In [1]:
import argparse
import qlib
import ruamel.yaml as yaml
from qlib.utils import init_instance_by_config
import pandas as pd
import torch
from qlib.contrib.evaluate import (
    backtest as normal_backtest,
    risk_analysis,
)

In [2]:
parser = argparse.ArgumentParser(allow_abbrev=False)
parser.add_argument("--seed", type=int, default=1000, help="random seed")
parser.add_argument("--config_file", type=str, default="us_latest.yaml", help="config file")
parser.add_argument("-f")
args = parser.parse_args()


seed = args.seed
config_file = args.config_file

# set random seed
with open(config_file) as f:
    config = yaml.safe_load(f)

# seed_suffix = "/seed1000" if "init" in config_file else f"/seed{seed}"
seed_suffix = ""
config["task"]["model"]["kwargs"].update(
    {"seed": seed, "logdir": config["task"]["model"]["kwargs"]["logdir"] + seed_suffix}
)

# initialize workflow
qlib.init(
    provider_uri=config["qlib_init"]["provider_uri"],
    region=config["qlib_init"]["region"],
)

[439:MainThread](2021-11-02 00:29:21,009) INFO - qlib.Initialization - [config.py:386] - default_conf: client.
[439:MainThread](2021-11-02 00:29:21,011) WARNING - qlib.Initialization - [config.py:411] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[439:MainThread](2021-11-02 00:29:21,182) INFO - qlib.Initialization - [__init__.py:56] - qlib successfully initialized based on client settings.
[439:MainThread](2021-11-02 00:29:21,182) INFO - qlib.Initialization - [__init__.py:58] - data_path={'__DEFAULT_FREQ': PosixPath('/home/us_data_dump2')}


In [3]:
latest_dataset = init_instance_by_config(config["task"]["dataset"])

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
[439:MainThread](2021-11-02 00:31:04,317) INFO - qlib.timer - [log.py:113] - Time cost: 103.122s | Loading data Done
[439:MainThread](2021-11-02 00:31:04,774) INFO - qlib.timer - [log.py:113] - Time cost: 0.314s | FilterCol Done
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.8/dist-p

In [4]:
model = init_instance_by_config(config["task"]["model"])

Please install necessary libs for CatBoostModel.
Please install necessary libs for XGBModel, such as xgboost.


[439:MainThread](2021-11-02 00:31:11,987) WARNING - qlib.TRA - [pytorch_tra.py:100] - `eval_train` will be ignored when using TRA.router
[439:MainThread](2021-11-02 00:31:11,990) WARNING - qlib.TRA - [pytorch_tra.py:132] - logdir output/nasdaq is not empty
[439:MainThread](2021-11-02 00:31:11,991) INFO - qlib.TRA - [pytorch_tra.py:141] - init TRAModel...
[439:MainThread](2021-11-02 00:31:11,995) WARNING - qlib.TRA - [pytorch_tra.py:150] - load state dict from `init_state`
[439:MainThread](2021-11-02 00:31:11,998) WARNING - qlib.TRA - [pytorch_tra.py:154] - {'unexpected_keys': [], 'missing_keys': [], 'error_msgs': []}
[439:MainThread](2021-11-02 00:31:11,999) INFO - qlib.TRA - [pytorch_tra.py:171] - # model params: 59520
[439:MainThread](2021-11-02 00:31:12,000) INFO - qlib.TRA - [pytorch_tra.py:172] - # tra params: 5606


RNN(
  (rnn): LSTM(20, 64, num_layers=2, batch_first=True)
  (W): Linear(in_features=64, out_features=64, bias=True)
  (u): Linear(in_features=64, out_features=1, bias=False)
)
TRA(
  (predictors): Linear(in_features=128, out_features=3, bias=True)
  (router): LSTM(3, 32, batch_first=True)
  (fc): Linear(in_features=160, out_features=3, bias=True)
)


In [40]:
state_dict = torch.load("/home/qlib_test/examples/benchmarks/TRA/output/us_158_nassp_epoch100/model.bin", map_location="cpu")["model"]

In [41]:
model.model.load_state_dict(state_dict)

<All keys matched successfully>

In [42]:
model.fitted = True

In [43]:
pred = model.predict(latest_dataset)

100%|█████████████████████████████████████████████| 1250/1250 [02:48<00:00,  7.41it/s]
[439:MainThread](2021-11-02 09:57:05,507) INFO - qlib.TRA - [pytorch_tra.py:516] - test metrics: {'MSE': 0.16629821699082611, 'MAE': 0.33140974398651785, 'IC': 0.001619795364845098, 'ICIR': 0.01216679586076465}


In [44]:
reset_df = pred.reset_index('instrument')

In [45]:
import qlib
from qlib.data import D
qlib.init(provider_uri="/home/us_data_dump2", region="us")

df = D.features(D.instruments("nassp"), ["$open", "$factor"], freq="day")

past_data = df.reset_index('instrument')

[439:MainThread](2021-11-02 10:03:21,353) INFO - qlib.Initialization - [config.py:386] - default_conf: client.
[439:MainThread](2021-11-02 10:03:21,355) WARNING - qlib.Initialization - [config.py:411] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[439:MainThread](2021-11-02 10:03:21,358) INFO - qlib.Initialization - [__init__.py:56] - qlib successfully initialized based on client settings.
[439:MainThread](2021-11-02 10:03:21,359) INFO - qlib.Initialization - [__init__.py:58] - data_path={'__DEFAULT_FREQ': PosixPath('/home/us_data_dump2')}


In [78]:
import time
import numpy as np

def backtest(top_k, max_buying, fee, p=False, early_stop=True, money=1000.0):
    st = time.time()
    indexs = reset_df.index.drop_duplicates()
    cnt = 0 
    # money = 100
    current_buying = {}
    num_buy = 0
    loss_cut = 0.95
    my_stack = None
    win, lose, same = 0, 0, 0
    past_money = money
    for ix, idx in enumerate(indexs):
        cnt+=1
        today_buy_list = []
        today_sell_list = []
        today_earn = 0
        
        try:
            if p:
                print("---------------------------------")
                print("day ", cnt, idx)
                print("---------------------------------")
                
            ## predict today score
            if type(reset_df.loc[idx]['score']) is np.float32:
                stack = reset_df.loc[idx]
            else:
                stack = reset_df.loc[idx].sort_values(by='score' , ascending=False)
                # stack = stack[stack['score'] < -0.17]
            
            ## update current stocks' prices, money
            # print("money, can_buy_money:", money, can_buy_money)
            can_buy_money = money
            current_buying_list = list(current_buying.keys())
            for cur_stock in current_buying_list:
                today_stock_price = df.loc[cur_stock,indexs[ix+1]][0] / df.loc[cur_stock,indexs[ix+1]][1]
                today_stock_price = today_stock_price.round(2)
                current_buying[cur_stock][2] = today_stock_price.round(2)
                current_buying[cur_stock][4] = (current_buying[cur_stock][3] * today_stock_price).round(2)
                ### update current total money
                if not np.isnan(current_buying[cur_stock][4]):
                    can_buy_money += current_buying[cur_stock][4].round(2)
            # print("money, can_buy_money:", money, can_buy_money)
            
            if p:
                print("current stocks: ", current_buying)
                
            ## sorting current stocks' score
            my_stack = stack[stack['instrument'].isin(current_buying_list)]
            my_stack = my_stack.sort_values(by='score' , ascending=True)
            my_stack_list = list(my_stack['instrument'])
            
            # print("current stocks' score:", my_stack_list)
            ## select buy_list and sell_list
            today_buy_cnt = 0
            for x in stack['instrument']:
                if today_buy_cnt >= top_k:
                    break
                cur_stock = x
                
                if cur_stock in my_stack_list:
                    # today_buy_list.append(cur_stock)
                    # today_sell_list.append(cur_stock)
                    today_buy_cnt+=1
                    if not np.isnan(current_buying[cur_stock][4]):
                        can_buy_money -= current_buying[cur_stock][4].round(2)
                    continue
                
                ### assert we can buy this stock
                today_stock_price = df.loc[cur_stock,indexs[ix+1]][0] / df.loc[cur_stock,indexs[ix+1]][1]
                today_stock_price = today_stock_price.round(2)
                
                if today_stock_price > can_buy_money / (max_buying - today_buy_cnt):
                    continue
                ### assert price is not nan
                if np.isnan(today_stock_price):
                    continue
                
                ### sell
                if num_buy >= max_buying:
                    last_stock = my_stack_list[-1]
                    num_last_stock = current_buying[last_stock][0]

                    num_buy -= 1
                    today_sell_list.append(last_stock)
                    if num_last_stock == 1:
                        earn = current_buying[last_stock][4]
                        del current_buying[last_stock]
                        my_stack_list = my_stack_list[:-1]
                    else:
                        current_buying[last_stock][0] -= 1
                        current_buying[last_stock][3] -= sell_total
                        current_buying[last_stock][4] -= sell_total*end
                        earn = sell_total*end
                    if not np.isnan(earn):
                        money += earn.round(2)*(1.0-fee)

                ### buy
                total = 1
                while(1):
                    total_price = (total+1)*today_stock_price
                    if total_price < can_buy_money / (max_buying - today_buy_cnt):
                        total+=1
                    else:
                        break
                total_price = total*today_stock_price
                total_price = total_price.round(2)
                
                num_buy+=1
                today_buy_cnt += 1
                today_buy_list.append(cur_stock)
                if cur_stock not in current_buying:
                    current_buying[x] = [1, today_stock_price, today_stock_price, total, total_price] #num, start_price, current_price, total, total_price
                else:
                    current_buying[x][0] += 1
                    current_buying[x][3] += total
                    current_buying[x][4] += total_price
                if not np.isnan(total_price):
                    money -= total_price.round(2)*(1.0+fee)
                    can_buy_money -= total_price.round(2)*(1.0+fee)
            if p:
                print("buy: ", today_buy_list)
                print("sell: ", today_sell_list)
            # print("current stocks: ", current_buying)
            ## caluate current_money
            current_money = money
            for c in current_buying:
                total_price = current_buying[c][4]
                current_money += total_price
            # if money<0:
                # print("?")
            if current_money > past_money:
                win += 1
            elif current_money == past_money:
                same += 1
            else:
                lose += 1
            
                
            past_money = current_money
            if p:
                print("current stocks: ", len(current_buying), current_buying)
                print("current usable money: ", money)
                print("current money: ", current_money)
        except Exception as e:
            print(e)
            if early_stop and cnt >= 800:
                print("---------------------------------")
                print("config: ", top_k, max_buying)
                print("day ", cnt, idx)
                print("current stocks: ", len(current_buying), current_buying)
                print("current money: ", current_money)
                print("time:", time.time() - st)
                break
            continue
        if early_stop and cnt >= 800:
            print("---------------------------------")
            print("config: ", top_k, max_buying)
            print("day ", cnt, idx)
            print("current stocks: ", len(current_buying), current_buying)
            print("current usable money: ", money)
            print("current money: ", current_money)
            print("time:", time.time() - st)
            break
    print(win, lose, same)
    return current_money

In [81]:
backtest(1, 1, 0.0000, money=10000, p=True, early_stop = False)

---------------------------------
day  1 2016-11-11 00:00:00
---------------------------------
current stocks:  {}
buy:  ['PGR']
sell:  []
current stocks:  1 {'PGR': [1, 33.29, 33.29, 300, 9987.0]}
current usable money:  13.0
current money:  10000.0
---------------------------------
day  2 2016-11-14 00:00:00
---------------------------------
current stocks:  {'PGR': [1, 33.29, 33.37, 300, 10011.0]}
buy:  ['FSLR']
sell:  ['PGR']
current stocks:  1 {'FSLR': [1, 33.16, 33.16, 302, 10014.32]}
current usable money:  9.680000000000291
current money:  10024.0
---------------------------------
day  3 2016-11-15 00:00:00
---------------------------------
current stocks:  {'FSLR': [1, 33.16, 33.03, 302, 9975.06]}
buy:  ['APD']
sell:  ['FSLR']
current stocks:  1 {'APD': [1, 138.08, 138.08, 72, 9941.76]}
current usable money:  42.97999999999956
current money:  9984.74
---------------------------------
day  4 2016-11-16 00:00:00
---------------------------------
current stocks:  {'APD': [1, 138.08

21700.39999999996

In [51]:
backtest(1, 1, 0.0000, money=10000, p=False, early_stop = False)

index 1250 is out of bounds for axis 0 with size 1250


21700.39999999996